##### Students :
- Adrien Mainka 20130281
- Abdelmoneim AlKhalil

To be able to run this notebook, the data files needs to be unzipped and store in `./data`

## Vocabulary Size

To measure the vocabulary size, we use a simple naive space tokenizer :<br> `tr ' ' '\n'`

In [3]:
import time
from collections import Counter
from itertools import chain

def countInFile(filename):
    with open(filename) as f:
        return Counter(chain.from_iterable(map(str.split, f)))

### train.en

In [4]:
# !cat data/train.en | tr ' ' '\n' | sort | uniq | wc -w
# 859973
start_time = time.time()
train_en = countInFile("data/train.en")
print("vocab size = {:,}\t elapsed time = {:.2f} s".format(len(train_en), time.time() - start_time))

vocab size = 860,818	 elapsed time = 24.27 s


### train.fr

In [5]:
# !cat data/train.fr | tr ' ' '\n' | sort | uniq | wc -w
# TOO SLOW

start_time = time.time()
train_fr = countInFile("data/train.fr")
print("vocab size = {:,}\t elapsed time = {:.2f} s".format(len(train_fr), time.time() - start_time))

vocab size = 3,310,267	 elapsed time = 339.34 s


### train.de

In [6]:
# !cat data/train.de | tr ' ' '\n' | sort | uniq | wc -w
start_time = time.time()
train_de = countInFile("data/train.de")
print("vocab size = {:,}\t elapsed time = {:.2f} s".format(len(train_de), time.time() - start_time))

vocab size = 1,713,522	 elapsed time = 26.84 s


## Number of words seen at least 100 times
### train.en

In [8]:
# !cat data/train.en | tr ' ' '\n' | sort | uniq -c | sort -k1 -nr | awk '$1 > 99' | wc -l
print("Number of words seenat least 100 times : {:,}".format(len([token for token in train_en if train_en[token] >= 100])))

Number of words seenat least 100 times : 30,395


## train.fr

In [9]:
# !cat data/train.fr | tr ' ' '\n' | sort | uniq -c | sort -k1 -nr | awk '$1 > 99' | wc -l
print("Number of words seenat least 100 times : {:,}".format(len([token for token in train_fr if train_fr[token] >= 100])))

Number of words seenat least 100 times : 124,291


### train.de

In [10]:
# !cat data/train.de | tr ' ' '\n' | sort | uniq -c | sort -k1 -nr | awk '$1 > 99' | wc -l
print("Number of words seenat least 100 times : {:,}".format(len([token for token in train_de if train_de[token] >= 100])))

Number of words seenat least 100 times : 42,518
